In [ ]:
import sys
sys.path.append('../keras_test')

In [ ]:
batch_size = 128
n_classes = 10


In [ ]:
from load_mnist import load_mnist
(x_train, y_train), (x_test, y_test) = load_mnist(x_style='one_channel')


In [ ]:
x_train.shape

In [ ]:
epochs = 3
log_filepath = './log'


In [ ]:
from keras.models import Model, Sequential
from keras.engine.topology import Input
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam
import keras.callbacks
from tensorboard_environment import TensorboardEnvironment
from my_model import simple_full_connected, vgg_like
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

with TensorboardEnvironment():
    input_shape = x_train.shape[1:]
    input_tensor = Input(shape=input_shape)
    x = Flatten(name='flatten')(input_tensor)
    x = Dense(1024, name='dense1', kernel_initializer='normal', kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01))(x)
    x = Activation('relu', name='relu1')(x)
    x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(x)
    x = Dropout(0.0, name='dropout1')(x)
    x = Dense(1024, name='dense2', kernel_initializer='normal', kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01))(x)
    x = Activation('relu', name='relu2')(x)
    x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(x)
    x = Dropout(0.0, name='dropout2')(x)
    x = Dense(n_classes, name='dense3', kernel_initializer='normal') (x)
    x = Activation('softmax', name='softmax1')(x)
    model = Model(inputs=input_tensor, outputs=x)
    model.summary()
    optimizer = SGD(lr=0.01, momentum=0.1, decay=0.0, nesterov=False)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_filepath, histogram_freq=1)
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[tensorboard_callback], validation_split=0.1)
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy;', score[1])
